In [1]:
import pandas as pd

import tqdm

import pickle

In [2]:
df = pd.read_csv('task.csv', delimiter=';')

print(df.head)


<bound method NDFrame.head of                                 house_uuid   latitude  longitude  \
0     b885de0f-09df-441f-85a6-bb28b468cea2  54.280637  48.310075   
1     ac845abb-39e4-4b59-b1ca-d01a6b883b6a  54.327441  48.388516   
2     fcbdc6c9-8c50-4a9d-9f2a-3da4f0d8432b  54.333380  48.491670   
3     837ff3e3-6b90-459b-926d-230dcf8a7169  53.650325  47.114310   
4     00588c27-989b-4d05-bf28-872c37831c9f  54.273530  48.286130   
...                                    ...        ...        ...   
5750  c4e0ce30-df67-49e3-a9ab-b54264293bf7  54.226301  48.093087   
5751  7992d45d-2e74-4f4b-b7c0-85ce5a7d8056  54.277110  48.282410   
5752  41924162-917f-45c4-b2dd-2c2e5069cfce  54.327908  48.486855   
5753  8b62ab41-dc72-4bfe-aa2f-6f564d7e0b07  54.275023  48.331051   
5754  2c4ba761-efb9-4401-84e3-d87ae6b46cf0  54.423633  48.171412   

      end_devices_count  
0                    60  
1                    35  
2                    38  
3                    22  
4                   101

In [3]:
df.shape

(5755, 4)

In [4]:
class bs1:
    radius = 1000
    free_users = 2000
    capacity = 2000
    cost = 50000
    
class bs2:
    radius = 2000
    free_users = 10000
    capacity = 10000
    cost = 180000

In [27]:
from geopy.distance import great_circle as GRC
from math import radians, cos, sin, asin, sqrt

class House:
    def __init__(self, uuid, lat, lon, count, has_bs):
        self.uuid = uuid
        self.lat = lat
        self.lon = lon
        self.count = count
        self.not_connected = count
        self.bs = has_bs
        
    def place_station(self, station, city):
        if station == 'bs1':
            self.bs = bs1()
        if station == 'bs2':
            self.bs = bs2()
        for i in self.closest:
            #print('tut', i)
            if self.distance_d(city.get_by_uuid(i)) > self.bs.radius:
                break
            if self.bs.free_users > 0:
                '''
                if self.bs.free_users >= self.not_connected:
                    self.bs.free_users -= self.not_connected
                    self.not_connected = 0
                elif self.bs.free_users < self.not_connected:
                    self.not_connected -= self.bs.free_users
                    self.bs.free_users = 0
                '''
                if self.bs.free_users >= city.get_by_uuid(i).not_connected:
                    self.bs.free_users -= city.get_by_uuid(i).not_connected
                    city.get_by_uuid(i).not_connected = 0
                elif self.bs.free_users < city.get_by_uuid(i).not_connected:
                    city.get_by_uuid(i).not_connected -= self.bs.free_users
                    self.bs.free_users = 0
                
                
    def try_station(self, station):
        if station == 'bs1':
            bs = bs1()
        if station == 'bs2':
            bs = bs2()
        for i in self.closest:
            if self.distance_d(city.get_by_uuid(i)) > bs.radius:
                break
            if bs.free_users > 0:
                if bs.free_users >= city.get_by_uuid(i).not_connected:
                    bs.free_users -= city.get_by_uuid(i).not_connected
                elif bs.free_users < city.get_by_uuid(i).not_connected:
                    bs.free_users = 0
        connected = bs.capacity - bs.free_users
        return connected/bs.cost
            
        
    def find_neighbours(self, mas):
        dists = {}
        for i in range(len(mas)):
            dists.update({i:self.distance_d(mas[i])})
        closest = dict(sorted(dists.items(), key=lambda item: item[1]))
        tmp = []
        for i in closest:
            tmp.append(mas[i].uuid)
            #if len(tmp) == 11:
            #    break
                
        #у нас где-то несколько домов по одному адресу
        try:
            tmp.remove(self.uuid)
        except:
            #print(closest)
            pass
        
        self.closest = tmp
        return closest
    
    #реально расстояние но медленное
    def dist_in_m(self, target):
        c_1 = (self.lat, self.lon)
        c_2 = (target.lat, target.lon)

        return GRC(c_1,c_2).km * 1000
            
    #бесполезное расстояние
    def dist_to_house(self, target):
        from math import sqrt
        return sqrt((self.lat-target.lat)**2 + (self.lon-target.lon)**2)
    
    #расстояние без ООП
    def find_dist(self, to_house):
        from math import sqrt
        return sqrt((self.lat-to_house[1])**2 + (self.lon-to_house[2])**2)
    
    #находим расстояние реально в метрах и быстро
    def distance_d(self, target):
        LoA = self.lon 
        LoB = target.lon  
        LaA= self.lat  
        LaB = target.lat 
        # The function "radians" is found in the math module, It's also used to convert radians to degrees.  
        LoA = radians(LoA)  
        LoB = radians(LoB)  
        LaA= radians(LaA)  
        LaB = radians(LaB) 
        # The "Haversine formula" is used.
        D_Lo = LoB - LoA 
        D_La = LaB - LaA 
        P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  

        Q = 2 * asin(sqrt(P))   
            # The earth's radius in kilometers.
        R_km = 6371  
        # Then we'll compute the outcome.
        return(Q * R_km) * 1000


In [48]:
class City:
    def __init__(self, arr):
        self.houses = arr
        
    def get_by_uuid(self, uuid):
        for i in self.houses:
            if i.uuid == uuid:
                return i
            
    def get_percent(self):
        total = 1
        connected = 1
        for i in self.houses:
            total += i.count
            connected += i.count - i.not_connected
        return connected/total
    
    def place_by_num(self, num):
        while self.houses[num].bs != False:
            if num >= len(self.houses):
                num = 0
            num+=1        
        self.houses[num].place_station('bs1', self)
        return self.houses[num].uuid
    
    def place_by_uuid(self, uuid, station):
        for house in self.houses:
            if house.uuid == uuid:
                house.place_station(station, self)
                break

    ################
    def get_house_by_coords(self, coords):
        dists = {}
        for i in range(len(self.houses)):
            dists.update({i:self.get_distance(coords, self.houses[i])})
        closest = dict(sorted(dists.items(), key=lambda item: item[1]))
        for i in closest:
            house = self.houses[i]
            break
        return house.uuid
        
        #находим расстояние реально в метрах и быстро
    def get_distance(self, coords, target):
        LoA = coords[1] 
        LoB = target.lon  
        LaA= coords[0]  
        LaB = target.lat 
        # The function "radians" is found in the math module, It's also used to convert radians to degrees.  
        LoA = radians(LoA)  
        LoB = radians(LoB)  
        LaA= radians(LaA)  
        LaB = radians(LaB) 
        # The "Haversine formula" is used.
        D_Lo = LoB - LoA 
        D_La = LaB - LaA 
        P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  

        Q = 2 * asin(sqrt(P))   
            # The earth's radius in kilometers.
        R_km = 6371  
        # Then we'll compute the outcome.
        return(Q * R_km) * 1000

In [51]:
##############ыыыыыыыыыыыыыыыыыыыы

with open('mas.pkl', 'rb') as f:
    tmp = pickle.load(f)
city = City(tmp)

from sklearn.cluster import KMeans
import numpy as np

tmp = []

for house in city.houses:
    tmp.append([house.lat, house.lon])
    
arr = np.array(tmp)

kmeans = KMeans(n_clusters=260, random_state=0, n_init="auto").fit(arr)
#250 мало

centers = kmeans.cluster_centers_

result = []
for cent in tqdm.tqdm(centers, total=len(centers)):
    uuid = city.get_house_by_coords(cent)
    s1 = city.get_by_uuid(uuid).try_station('bs1')
    s2 = city.get_by_uuid(uuid).try_station('bs2')
    #print(sq1, sq2)
    if s1 < s2:  
        city.place_by_uuid(uuid, 'bs1')
        result.append([uuid, 1])
        
    elif s2>s1:
        city.place_by_uuid(uuid, 'bs2')
        result.append([uuid, 2])

        
        
print(city.get_percent())


result = pd.DataFrame(result, columns=['house_uuid', 'type'])
result.to_csv("K.csv", sep=';', index = False)

100%|████████████████████████████████████████████████████████████████████████████████| 260/260 [01:17<00:00,  3.37it/s]

0.7977881182855393


In [36]:
s1 = city.houses[100].try_station('bs1')
s2 = city.houses[100].try_station('bs2')
print(s1, s2)

0.04 0.05555555555555555


# generate data array

In [ ]:
mas = []

l_up = (54.48116, 48.148)
r_dw = (54.07614, 49)

for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0]):
    #if row['latitude'] <= l_up[0] and row['latitude'] >= r_dw[0] and row['longitude'] >= l_up[1] and row['longitude']<=r_dw[1]:
        mas.append([row['house_uuid'], row['latitude'], row['longitude'], row['end_devices_count']])
  
tmp_mas = mas.copy()


for index in range(len(mas)):
    row = mas[index]

    mas[index] = House(row[0], row[1], row[2], row[3], False)

    

# находим расстояние от каждого дома до каждого дома урааа
for house in tqdm.tqdm(mas, total = len(mas)):
    house.find_neighbours(mas)
        
    



 56%|███████████████████████████████████████████▌                                  | 3215/5755 [01:02<00:48, 52.89it/s]

In [8]:


with open('mas.pkl', 'wb') as f:
    pickle.dump(mas, f)
    

# Random solution

In [27]:
from random import randint

with open('mas.pkl', 'rb') as f:
    tmp = pickle.load(f)
    
    
city = City(tmp)
result = []
result_tmp = []

print(len(city.houses))

for i in tqdm.tqdm(range(1, 350)):
    h_uuid = city.place_by_num(randint(0, len(city.houses)-1))

    result.append([h_uuid, 1])
    result_tmp.append(h_uuid)
    
for i in result_tmp:
    if result_tmp.count(i) > 1:
        print(i)

    
    
#print(sorted(result))
print(city.get_percent())


result = pd.DataFrame(result, columns=['house_uuid', 'type'])
result.to_csv("Random.csv", sep=';', index = False)

3537


100%|████████████████████████████████████████████████████████████████████████████████| 349/349 [00:27<00:00, 12.85it/s]

0.9487875698573077


In [51]:
arr

array([[54.280637, 48.310075],
       [54.327441, 48.388516],
       [54.33338 , 48.49167 ],
       ...,
       [54.327908, 48.486855],
       [54.275023, 48.331051],
       [54.423633, 48.171412]])

# Check solution

In [22]:


city = City(mas.copy())
eval_pos = pd.read_csv('result-example.csv', delimiter=';')

for index, row in tqdm.tqdm(eval_pos.iterrows()):
    if row['type'] == 1:
        h = city.get_by_uuid(row['house_uuid'])
        if h == None:
            continue
        #print(row['house_uuid'])
        h.place_station('bs1')
    
    elif row['type'] == 2:
        city.get_by_uuid(row['house_uuid']).place_station('bs2')

    
    
city.get_percent()

29it [00:00, 926.47it/s]


0.7419931160480867

In [ ]:
import pytorch
import torch.nn as nn


class hz_model(nn.Module):
    def __init__(self, types):
        super().__init__()
        self.dropout = nn.Dropout(0.25)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,types)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [8]:
from ipyleaflet import Map, Marker, Circle, CircleMarker, AwesomeIcon, Heatmap,DivIcon, MarkerCluster


#      latitude    longitude
center = (54.33338, 48.49167)

m = Map(center=center, zoom=15)

markers = []

for house in tqdm.tqdm(mas):
    markers.append(Marker(location=(house.lat, house.lon), title=str(house.not_connected)))
    
marker_cluster = MarkerCluster(
    markers=markers
)

m.add_layer(marker_cluster);

m

100%|█████████████████████████████████████████████████████████████████████████████| 3537/3537 [00:04<00:00, 858.78it/s]


Map(center=[54.33338, 48.49167], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

No such comm: c2915cfbaacb425989dffc008b82b3ab
No such comm: c2915cfbaacb425989dffc008b82b3ab
No such comm: c2915cfbaacb425989dffc008b82b3ab
No such comm: c2915cfbaacb425989dffc008b82b3ab


In [ ]:
'''import folium
from folium.features import DivIcon

m = folium.Map(location=(54.33338, 48.49167))

for index, row in df.iterrows():
    p1 = (row['latitude'], row['longitude'])
    folium.Marker(p1, icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 18pt; color : black">'+str(row['end_devices_count'])+'</div>',
        )).add_to(m)
    m.add_child(folium.CircleMarker(p1, radius=15))
    
    

m
''''''